In [1]:
import os
import zipfile
from pathlib import Path
import gcsfs
import google.auth
from io import BytesIO
import requests
from google.cloud import storage

In [ ]:
# !gcloud init


Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
core:
  account: stockumlars@gmail.com
  project: glowing-carport-426208-n2

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  

Command killed by keyboard interrupt

^C


In [6]:
# Function to get gcsfs client
def get_gcsfs(project_name):
    creds, project = google.auth.default()
    gcs = gcsfs.GCSFileSystem(token=creds, project=project_name)
    return gcs

# # Function to download and extract datasets
# def download_and_extract(url, zip_dir, extract_dir):
#     if not extract_dir.exists():
#         if not zip_dir.exists():
#             !wget -O {zip_dir} {url}
#         with zipfile.ZipFile(zip_dir, "r") as zip_ref:
#             zip_ref.extractall()
#         zip_dir.unlink()

# # Function to upload files to GCS using gcsfs
# def upload_to_gcsfs(local_path, gcsfs, bucket_name, gcs_path):
#     for file in local_path.rglob('*'):
#         if file.is_file():
#             destination_path = f'{bucket_name}/{gcs_path}/{file.relative_to(local_path)}'
#             with gcsfs.open(destination_path, 'wb') as f:
#                 f.write(file.read_bytes())

# # Function to count number of files in each directory
# def count_files_in_directories(paths):
#     file_counts = {}
#     for path in paths:
#         if path.exists() and path.is_dir():
#             file_counts[path.name] = len(list(path.iterdir()))
#         else:
#             file_counts[path.name] = 0
#     return file_counts

In [7]:
google.colab.auth.authenticate_user()

In [ ]:
def upload_to_gcsfs(local_path, gcsfs, bucket_name, gcs_path):
    print(local_path)
    for file in local_path.rglob('*'):
        if file.is_file():
            destination_path = f'{bucket_name}/{gcs_path}/{file.relative_to(local_path)}'
            print(f'Uploading {file} to {destination_path}')
            with gcsfs.open(destination_path, 'wb') as f:
                f.write(file.read_bytes())


In [ ]:
# def download_and_extract_to_gcs(url, bucket_name, gcs_path, gcsfs):
#     print("Downloading:", url)
#     response = requests.get(url)
#     with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
#         for zip_info in zip_ref.infolist():
#             if not zip_info.is_dir():
#                 file_data = zip_ref.read(zip_info.filename)
#                 destination_path = f'{bucket_name}/{gcs_path}/{zip_info.filename}'
#                 print(f'Uploading {zip_info.filename} to {destination_path}')
#                 with gcsfs.open(destination_path, 'wb') as f:
#                     f.write(file_data)
# Function to download, extract datasets directly to GCS, and remove zip
# def download_extract_upload_gcs(url, bucket_name, gcs_path, gcsfs):
#     # Download the file
#     zip_file_path = Path(f'{gcs_path}.zip')
#     !wget -O {zip_file_path} {url}

#     # Write the zip file to disk
#     #with open(zip_file_path, 'wb') as f:
#     #    f.write(response.content)

#     # Extract the zip file and upload to GCS
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         for zip_info in zip_ref.infolist():
#             if not zip_info.is_dir():
#                 file_data = zip_ref.read(zip_info.filename)
#                 destination_path = f'{bucket_name}/{gcs_path}/{zip_info.filename}'
#                 print(f'Uploading {zip_info.filename} to {destination_path}')
#                 with gcsfs.open(destination_path, 'wb') as f:
#                     f.write(file_data)

#     # Remove the zip file after extraction
#     zip_file_path.unlink()


def count_files_in_gcs(gcsfs, bucket_name, directories):
    file_counts = {}
    for directory in directories:
        files = gcsfs.glob(f'{bucket_name}/{directory}/**')
        file_counts[directory] = len(files)
    return file_counts

In [ ]:
def download_and_extract_to_gcs(url, bucket_name, gcs_path, gcsfs):
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        for zip_info in zip_ref.infolist():
            if not zip_info.is_dir():
                file_data = zip_ref.read(zip_info.filename)
                destination_path = f'{bucket_name}/{gcs_path}/{zip_info.filename}'
                #print(f'Uploading {zip_info.filename} to {destination_path}')
                try:
                    with gcsfs.open(destination_path, 'wb') as f:
                        f.write(file_data)
                except Exception as e:
                    print(f'Error uploading {zip_info.filename}: {e}')

# # Define URLs and paths for datasets
# tallyqa = "https://github.com/manoja328/tallyqa/blob/master/tallyqa.zip?raw=true"
#tallyqa_dir = Path(".")
# tallyqa_zip_dir = Path("tallyqa.zip")

# # train = "http://images.cocodataset.org/zips/train2014.zip"
# # train_dir = Path("train2014")
# # train_zip_dir = Path("train2014.zip")

# # val = "http://images.cocodataset.org/zips/val2014.zip"
# # val_dir = Path("val2014")
# # val_zip_dir = Path("val2014.zip")

# # test = "http://images.cocodataset.org/zips/test2015.zip"
# # test_dir = Path("test2015")
# # test_zip_dir = Path("test2015.zip")

# # # Download and extract datasets
# #download_and_extract(tallyqa, tallyqa_zip_dir, tallyqa_dir)


# # Upload datasets to GCS
#upload_to_gcsfs(tallyqa_dir, gcs, bucket_name, 'tallyqa')


In [8]:
bucket_name = "multimodalai" #@param {type:"string"}
project_name = "glowing-carport-426208-n2" #@param {type:"string"}

# # Get the gcsfs client
gcs = get_gcsfs(project_name)

In [ ]:
gcs

In [ ]:
def upload_to_gcs(zip_file_url, bucket_name, zip_dir, destination_blob_name):
    #response = requests.get(zip_file_url)
    #if response.status_code == 200:

    !wget -O $zip_dir $zip_file_url
    # Initialize a client
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Upload the zip file to GCS
    blob.upload_from_filename(zip_dir)
    print(f"File {destination_blob_name} uploaded to {bucket_name}.")
    os.remove(zip_dir)


In [ ]:
datasets = {
    "tallyqa": "https://github.com/manoja328/tallyqa/blob/master/tallyqa.zip?raw=true",
    "train2014": "http://images.cocodataset.org/zips/train2014.zip",
    #"val2014": "http://images.cocodataset.org/zips/val2014.zip",
    #"test2015": "http://images.cocodataset.org/zips/test2015.zip"
}

# Upload datasets to GCS
for name, url in datasets.items():
    upload_to_gcs(url, bucket_name, name, f"{name}/{name}.zip")

--2024-06-12 12:04:42--  https://github.com/manoja328/tallyqa/blob/master/tallyqa.zip?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/manoja328/TallyQA_dataset/blob/master/tallyqa.zip?raw=true [following]
--2024-06-12 12:04:42--  https://github.com/manoja328/TallyQA_dataset/blob/master/tallyqa.zip?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/manoja328/TallyQA_dataset/raw/master/tallyqa.zip [following]
--2024-06-12 12:04:42--  https://github.com/manoja328/TallyQA_dataset/raw/master/tallyqa.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manoja328/TallyQA_dataset/master/tallyqa.zip [following]
--2024-06-12 12:04:42--  https://raw.githu

In [ ]:
# # Define URLs and paths for datasets
# tallyqa = "https://github.com/manoja328/tallyqa/blob/master/tallyqa.zip?raw=true"
# train = "http://images.cocodataset.org/zips/train2014.zip"
# val = "http://images.cocodataset.org/zips/val2014.zip"
# test = "http://images.cocodataset.org/zips/test2015.zip"

# # Download and extract datasets directly to GCS
# download_extract_upload_gcs(tallyqa, bucket_name, 'tallyqa', gcs)
# download_extract_upload_gcs(train, bucket_name, 'train2014', gcs)
# download_extract_upload_gcs(val, bucket_name, 'val2014', gcs)
# download_extract_upload_gcs(test, bucket_name, 'test2015', gcs)

# # List of directories to check
# directories_to_check = ['tallyqa', 'train2014', 'val2014', 'test2015']

# # Print the number of files in each directory
# file_counts = count_files_in_gcs(gcs, bucket_name, directories_to_check)
# for directory, count in file_counts.items():
#     print(f"Number of files in {directory}: {count}")

--2024-06-12 10:33:15--  https://github.com/manoja328/tallyqa/blob/master/tallyqa.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/manoja328/TallyQA_dataset/blob/master/tallyqa.zip?raw=true [following]
--2024-06-12 10:33:15--  https://github.com/manoja328/TallyQA_dataset/blob/master/tallyqa.zip?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/manoja328/TallyQA_dataset/raw/master/tallyqa.zip [following]
--2024-06-12 10:33:15--  https://github.com/manoja328/TallyQA_dataset/raw/master/tallyqa.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manoja328/TallyQA_dataset/master/tallyqa.zip [following]
--2024-06-12 10:33:15--  https://raw.githu

KeyboardInterrupt: 

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  13104      0 --:--:-- --:--:-- --:--:-- 13163
OK
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.7 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../gcsfuse_2.2.0_amd64.deb ...
Unpacking gcsfuse (2.2.0) ...
Setting up gcsfuse (2.2.0) ...


In [9]:
!mkdir folderOnColab
!gcsfuse multimodalai folderOnColab

{"timestamp":{"seconds":1718374182,"nanos":199549338},"severity":"INFO","message":"Start gcsfuse/2.2.0 (Go version go1.22.3) for app \"\" using mount point: /content/folderOnColab\n"}
{"timestamp":{"seconds":1718374182,"nanos":222490349},"severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":false,\"OnlyDir\":\"\",\"RenameDirLimit\":0,\"IgnoreInterrupts\":false,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"AnonymousAccess\":false,\"MaxRetrySleep\":30000000000,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"KernelListCacheTtlSeconds\":0,\"HttpClientTimeout\":0,\"MaxRetryDuration\":-1000000000,\"RetryMultiplier\":2,\"LocalFileCache\":false,\"

In [ ]:
import os
import zipfile
import os.path

# Ensure the target directory exists
target_dir = "/content/folderOnColab"
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Function to download and extract datasets using wget
def download_and_extract(url, zip_path, extract_dir):
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)
        # Download the file
        !wget -O $zip_path $url
        # Extract the file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        # Remove the zip file to save space
        os.remove(zip_path)
        print(f"Extracted {zip_path} to {extract_dir}")
    else:
        print(f"{extract_dir} already exists, skipping download.")

# Download and extract tallyqa dataset
tallyqa_url = "https://github.com/manoja328/tallyqa/blob/master/tallyqa.zip?raw=true"
download_and_extract(tallyqa_url, "tallyqa.zip", target_dir)

# URLs for COCO datasets
coco_datasets = {
    #"train2014": "http://images.cocodataset.org/zips/train2014.zip",
    "val2014": "http://images.cocodataset.org/zips/val2014.zip",
    #"test2015": "http://images.cocodataset.org/zips/test2015.zip"
}

# Download and extract COCO datasets into the target directory
for name, url in coco_datasets.items():
    zip_path = f"{name}.zip"
    extract_dir = os.path.join(target_dir,name)
    download_and_extract(url, zip_path, extract_dir)

# List the contents of the folderOnColab directory
!ls /content/folderOnColab


/content/folderOnColab already exists, skipping download.
--2024-06-14 15:47:27--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.53.233, 54.231.165.201, 54.231.203.41, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.53.233|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[===================>]   6.19G  70.4MB/s    in 1m 54s  

2024-06-14 15:49:21 (55.8 MB/s) - ‘val2014.zip’ saved [6645013297/6645013297]



In [23]:
%mv /train2014 /content/folderOnColab

^C


In [16]:
# https://stackoverflow.com/questions/61600439/how-to-mount-gcp-bucket-in-google-colab

bin			    datalab  kaggle  libx32		       opt   sbin  tools
boot			    dev      lib     media		       proc  srv   usr
content			    etc      lib32   mnt		       root  sys   var
cuda-keyring_1.0-1_all.deb  home     lib64   NGC-DL-CONTAINER-LICENSE  run   tmp
